In [1]:
# Imports
from extremeweatherbench import cases, inputs, metrics, evaluate, utils
import logging
import sys
# Set up logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger('botocore.httpchecksum').setLevel(logging.WARNING)

print("✓ Imports successful!")

✓ Imports successful!


In [2]:
model = 'FOUR_v200_GFS'
forecast_dir = f'gs://extremeweatherbench/{model}.parq'

fcnv2_forecast = inputs.KerchunkForecast(
    name="fcnv2_forecast",  # ← ADDED: Required parameter!
    source=forecast_dir,
    variables=["surface_air_temperature"],
    variable_mapping=inputs.CIRA_metadata_variable_mapping,
    storage_options={"remote_protocol": "s3", "remote_options": {"anon": True}},
)

print(f"✓ Forecast configured: {model}")

✓ Forecast configured: FOUR_v200_GFS


In [3]:
# Define ERA5 target (observations)
era5_heatwave_target = inputs.ERA5(
    variables=["surface_air_temperature"],
    storage_options={"remote_options": {"anon": True}},
    chunks=None,
)

print("✓ ERA5 target configured!")

✓ ERA5 target configured!


In [4]:
# Create evaluation object
heatwave_evaluation_list = [
    inputs.EvaluationObject(
        event_type="heat_wave",
        metric_list=[
            metrics.MaximumMeanAbsoluteError(),
            metrics.RootMeanSquaredError(),
            metrics.MaximumLowestMeanAbsoluteError(),
        ],
        target=era5_heatwave_target,
        forecast=fcnv2_forecast,
    ),
]

print("✓ Evaluation object created!")

✓ Evaluation object created!


In [5]:
# Load case metadata
case_metadata = cases.load_ewb_events_yaml_into_case_collection()

print(f"✓ Case metadata loaded!")
print(f"Type: {type(case_metadata)}")
print(case_metadata)  # Let's see what it looks like

✓ Case metadata loaded!
Type: <class 'extremeweatherbench.cases.IndividualCaseCollection'>
IndividualCaseCollection(cases=[IndividualCase(case_id_number=1, title='2021 Pacific Northwest', start_date=datetime.datetime(2021, 6, 20, 0, 0), end_date=datetime.datetime(2021, 7, 3, 0, 0), location=BoundingBoxRegion(latitude_min=33.25, latitude_max=68.0, longitude_min=215.25, longitude_max=256.0), event_type='heat_wave'), IndividualCase(case_id_number=2, title='2022 Upper Midwest', start_date=datetime.datetime(2022, 5, 7, 0, 0), end_date=datetime.datetime(2022, 5, 17, 0, 0), location=BoundingBoxRegion(latitude_min=19.5, latitude_max=48.25, longitude_min=252.0, longitude_max=276.75), event_type='heat_wave'), IndividualCase(case_id_number=3, title='2022 California', start_date=datetime.datetime(2022, 6, 7, 0, 0), end_date=datetime.datetime(2022, 6, 15, 0, 0), location=BoundingBoxRegion(latitude_min=23.75, latitude_max=44.5, longitude_min=235.75, longitude_max=266.25), event_type='heat_wave'), In

In [6]:
# Filter to heat wave cases only (for faster testing)
case_metadata.select_cases(by="event_type", value="heat_wave", inplace=True)

print("✓ Filtered to heat wave cases")
print(case_metadata)

✓ Filtered to heat wave cases
IndividualCaseCollection(cases=[IndividualCase(case_id_number=1, title='2021 Pacific Northwest', start_date=datetime.datetime(2021, 6, 20, 0, 0), end_date=datetime.datetime(2021, 7, 3, 0, 0), location=BoundingBoxRegion(latitude_min=33.25, latitude_max=68.0, longitude_min=215.25, longitude_max=256.0), event_type='heat_wave'), IndividualCase(case_id_number=2, title='2022 Upper Midwest', start_date=datetime.datetime(2022, 5, 7, 0, 0), end_date=datetime.datetime(2022, 5, 17, 0, 0), location=BoundingBoxRegion(latitude_min=19.5, latitude_max=48.25, longitude_min=252.0, longitude_max=276.75), event_type='heat_wave'), IndividualCase(case_id_number=3, title='2022 California', start_date=datetime.datetime(2022, 6, 7, 0, 0), end_date=datetime.datetime(2022, 6, 15, 0, 0), location=BoundingBoxRegion(latitude_min=23.75, latitude_max=44.5, longitude_min=235.75, longitude_max=266.25), event_type='heat_wave'), IndividualCase(case_id_number=4, title='2022 Texas', start_date

In [7]:
# Create ExtremeWeatherBench instance
ewb_instance = evaluate.ExtremeWeatherBench(
    case_metadata=case_metadata,
    evaluation_objects=heatwave_evaluation_list,
)

print("✓ ExtremeWeatherBench ready!")

✓ ExtremeWeatherBench ready!


In [8]:
# Add this at the end to actually run it
print("Starting evaluation (this may take a few minutes)...")
heatwave_outputs = ewb_instance.run(
    parallel_config={'backend':'loky','n_jobs':4}  # Use fewer jobs for testing
)
print(f"✓ Results: {heatwave_outputs.shape}")
print(heatwave_outputs.head())

Starting evaluation (this may take a few minutes)...
INFO:extremeweatherbench.evaluate:Running ExtremeWeatherBench workflow...
INFO:extremeweatherbench.evaluate:Running case operators in parallel...


 11%|█         | 5/46 [01:01<03:49,  5.61s/tasks]Data input fcnv2_forecast for case 6 has no data for case time range 2022-05-17 00:00:00 to 2022-05-24 00:00:00.
Data input fcnv2_forecast for case 7 has no data for case time range 2023-11-18 00:00:00 to 2023-11-28 00:00:00.
 26%|██▌       | 12/46 [01:02<00:26,  1.29tasks/s]Data input fcnv2_forecast for case 12 has no data for case time range 2023-05-24 00:00:00 to 2023-06-01 00:00:00.
Data input fcnv2_forecast for case 14 has no data for case time range 2023-07-05 00:00:00 to 2023-07-27 00:00:00.
 43%|████▎     | 20/46 [01:04<00:07,  3.60tasks/s]Data input fcnv2_forecast for case 21 has no data for case time range 2022-07-15 00:00:00 to 2022-07-23 00:00:00.
Data input fcnv2_forecast for case 22 has no data for case time range 2022-06-09 00:00:00 to 2022-06-21 00:00:00.
 52%|█████▏    | 24/46 [01:05<00:05,  3.70tasks/s]Data input fcnv2_forecast for case 24 has no data for case time range 2022-04-24 00:00:00 to 2022-05-04 00:00:00.
Data 

✓ Results: (0, 9)
Empty DataFrame
Columns: [value, lead_time, init_time, target_variable, metric, forecast_source, target_source, case_id_number, event_type]
Index: []


In [9]:
heatwave_outputs.to_csv('heatwave_evaluation_results.csv')

In [8]:
ds = fcnv2_forecast.open_and_maybe_preprocess_data_from_source()

print("Forecast time range:")
print(f"  Start: {ds.time.values.min()}")
print(f"  End: {ds.time.values.max()}")
print(f"  Total timesteps: {len(ds.time)}")

INFO:fsspec.reference:Open lazy reference dict from URL gs://extremeweatherbench/FOUR_v200_GFS.parq
INFO:fsspec.reference:Open lazy reference dict from URL gs://extremeweatherbench/FOUR_v200_GFS.parq
INFO:fsspec.reference:Open lazy reference dict from URL gs://extremeweatherbench/FOUR_v200_GFS.parq
Forecast time range:
  Start: 2020-09-30T12:00:00.000000000
  End: 2020-10-10T12:00:00.000000000
  Total timesteps: 41


In [11]:
for case in case_metadata.cases:
    start = case.start_date
    end = case.end_date
    # Check if case overlaps with forecast range (Sept 30 - Oct 10, 2020)
    if start.year == 2020 and start.month in [9, 10]:
        print(f"{case.event_type}: {case.title} ({start} to {end})")

atmospheric_river: October 2020 Western Alps (2020-10-02 00:00:00 to 2020-10-04 00:00:00)
atmospheric_river: September 2020 Pacific Northwest (2020-09-23 00:00:00 to 2020-09-28 00:00:00)
atmospheric_river: September 2020 Pacific Northwest (2020-09-14 00:00:00 to 2020-09-19 00:00:00)
tropical_cyclone: Paulette (2020-09-05 00:00:00 to 2020-09-30 12:00:00)
tropical_cyclone: Sally (2020-09-09 18:00:00 to 2020-09-20 06:00:00)
tropical_cyclone: Teddy (2020-09-10 06:00:00 to 2020-09-26 06:00:00)
tropical_cyclone: Delta (2020-10-02 18:00:00 to 2020-10-13 18:00:00)
tropical_cyclone: Zeta (2020-10-22 12:00:00 to 2020-11-01 00:00:00)
tropical_cyclone: Eta (2020-10-29 18:00:00 to 2020-11-16 00:00:00)
tropical_cyclone: Noul (Leon) (2020-09-12 00:00:00 to 2020-09-21 00:00:00)
tropical_cyclone: Linfa (2020-10-05 06:00:00 to 2020-10-14 00:00:00)
tropical_cyclone: Molave (Quinta) (2020-10-19 18:00:00 to 2020-10-31 06:00:00)
tropical_cyclone: Goni (Rolly) (2020-10-23 06:00:00 to 2020-11-08 06:00:00)
sev

In [ ]:
# Check what attributes a case has
case = case_metadata.cases[0]
print([attr for attr in dir(case) if not attr.startswith('_')])     

['case_id_number', 'end_date', 'event_type', 'location', 'start_date', 'title']


In [13]:
from extremeweatherbench import cases, inputs, metrics, evaluate

# Load heat wave cases
case_metadata = cases.load_ewb_events_yaml_into_case_collection()
case_metadata.select_cases(by="event_type", value="heat_wave", inplace=True)
print(f"Heat wave cases: {len(case_metadata.cases)}")

# Use the HRES data (2016-2022) instead of the sample file
hres_forecast = inputs.ZarrForecast(
    name="hres_forecast",
    source="gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr",
    variables=["surface_air_temperature"],
    variable_mapping=inputs.HRES_metadata_variable_mapping,
)

era5_target = inputs.ERA5(
    variables=["surface_air_temperature"],
    storage_options={"remote_options": {"anon": True}},
    chunks=None,
)

heatwave_evaluation = [
    inputs.EvaluationObject(
        event_type="heat_wave",
        metric_list=[metrics.RootMeanSquaredError()],
        target=era5_target,
        forecast=hres_forecast,
    ),
]

ewb = evaluate.ExtremeWeatherBench(
    case_metadata=case_metadata,
    evaluation_objects=heatwave_evaluation,
)

results = ewb.run(parallel_config={'backend':'loky', 'n_jobs':4})
print(results)

Heat wave cases: 46
INFO:extremeweatherbench.evaluate:Running ExtremeWeatherBench workflow...
INFO:extremeweatherbench.evaluate:Running case operators in parallel...


 83%|████████▎ | 38/46 [06:48<00:05,  1.55tasks/s]Data input hres_forecast for case 81 has no data for case time range 2024-08-10 00:00:00 to 2024-08-16 00:00:00.
Data input hres_forecast for case 83 has no data for case time range 2024-06-20 00:00:00 to 2024-06-30 00:00:00.
100%|██████████| 46/46 [07:09<00:00,  9.34s/tasks]

        value        lead_time  init_time          target_variable  \
0    1.648361  0 days 00:00:00        NaN  surface_air_temperature   
1    1.544227  0 days 06:00:00        NaN  surface_air_temperature   
2    1.696581  0 days 12:00:00        NaN  surface_air_temperature   
3    1.561064  0 days 18:00:00        NaN  surface_air_temperature   
4    1.720501  1 days 00:00:00        NaN  surface_air_temperature   
..        ...              ...        ...                      ...   
692  3.710632  9 days 00:00:00        NaN  surface_air_temperature   
693  3.381790  9 days 06:00:00        NaN  surface_air_temperature   
694  3.314265  9 days 12:00:00        NaN  surface_air_temperature   
695  3.699797  9 days 18:00:00        NaN  surface_air_temperature   
696  3.604586 10 days 00:00:00        NaN  surface_air_temperature   

                   metric forecast_source target_source  case_id_number  \
0    RootMeanSquaredError   hres_forecast          ERA5               1   
1    Root